# Merge all data into giant df

In [1]:
import pandas as pd
import numpy as np

In [2]:
sales = pd.read_csv('./input/cleanedmonthly.csv')
item_em = pd.read_csv('./input/item_embeddings.csv')
shop_em = pd.read_csv('./input/shop_embeddings.csv')
test = pd.read_csv('./input/test.csv')

In [3]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1608224 entries, 0 to 1608223
Data columns (total 7 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   year                  1608224 non-null  int64  
 1   month                 1608224 non-null  int64  
 2   shop_id               1608224 non-null  int64  
 3   item_id               1608224 non-null  int64  
 4   date_block_num        1608224 non-null  int64  
 5   item_cnt_month        1608224 non-null  float64
 6   median_monthly_price  1608224 non-null  float64
dtypes: float64(2), int64(5)
memory usage: 85.9 MB


In [4]:
sales

,year,month,shop_id,item_id,date_block_num,item_cnt_month,median_monthly_price
0,2013,1,57,32,0,6.0,221.0
1,2013,1,57,33,0,3.0,347.0
2,2013,1,57,35,0,1.0,247.0
3,2013,1,57,43,0,1.0,221.0
4,2013,1,57,51,0,2.0,128.5
...,...,...,...,...,...,...,...
1608219,2015,10,59,22087,33,6.0,119.0
1608220,2015,10,59,22088,33,2.0,119.0
1608221,2015,10,59,22091,33,1.0,179.0
1608222,2015,10,59,22100,33,1.0,629.0


In [5]:
items_train = sales['item_id'].unique().tolist()
items_test = test['item_id'].unique().tolist()

In [6]:
len(items_test), len(items_train)

(5100, 21802)

### items_only in test

In [7]:
items_only_test = []
for item in items_test:
    if item not in items_train:
        items_only_test.append(item)
len(items_only_test)

363

In [8]:
items = items_train + items_only_test
len(items)

22165

In [9]:
shops = sales['shop_id'].unique().tolist()
len(shops)

56

In [10]:
s = []
i = []
for shop in shops:
        s = [*s, *(np.ones(len(items))*shop)]
        i = [*i, *(np.ones(1)*items)]
s = np.array(s).astype(np.int8)
i = np.array(i).astype(np.int16)

In [24]:
len(s), len(i)

(1241240, 1241240)

In [20]:
date_block_nums = range(35)
x = []
for date_block in date_block_nums:
    temp = pd.DataFrame({'shop_id': s , 'item_id': i})
    temp['date_block_num'] = date_block
    temp['month'] =  1 + date_block%12
    temp['year'] = 2013 + date_block//12
    x.append(temp)
df = pd.concat(x, ignore_index=True)

In [21]:
df.sort_values(['shop_id', 'item_id', 'date_block_num'], inplace=True, ignore_index=True)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43443400 entries, 0 to 43443399
Data columns (total 5 columns):
 #   Column          Dtype
---  ------          -----
 0   shop_id         int8 
 1   item_id         int16
 2   date_block_num  int64
 3   month           int64
 4   year            int64
dtypes: int16(1), int64(3), int8(1)
memory usage: 1.1 GB


In [23]:
df

,shop_id,item_id,date_block_num,month,year
0,2,0,0,1,2013
1,2,0,1,2,2013
2,2,0,2,3,2013
3,2,0,3,4,2013
4,2,0,4,5,2013
...,...,...,...,...,...
43443395,59,22169,30,7,2015
43443396,59,22169,31,8,2015
43443397,59,22169,32,9,2015
43443398,59,22169,33,10,2015


In [34]:
df.to_csv('./input/basestructuredf.csv', index=False)